# Vector Database Operations with pgvector

In [3]:
import os
import psycopg2
from docarray import BaseDoc, DocList
from docarray.typing import NdArray
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from pgvector.psycopg2 import register_vector


# Load environment variables from the .env file
load_dotenv()

# Database connection
DATABASE_URL = os.environ['DATABASE_URL']
conn = psycopg2.connect(DATABASE_URL)
register_vector(conn)
cur = conn.cursor()

In [76]:
# rollback db transaction if error occurs
#conn.rollback()

In [22]:
# Query drink data from drink tables

cur.execute("""
SELECT 
  d.id AS drink_id,
  d.name AS drink_name,
  STRING_AGG(DISTINCT t.name, ', ') AS tags,
  STRING_AGG(DISTINCT i.name, ', ') AS ingredients
FROM drinks d
LEFT JOIN drink_tags dt ON d.id = dt.drink_id
LEFT JOIN tags t ON dt.tag_id = t.id
LEFT JOIN drink_ingredients di ON d.id = di.drink_id
LEFT JOIN ingredients i ON di.ingredient_id = i.id
WHERE d.reference like 'http%'
GROUP BY d.id, d.name
ORDER BY d.name
""")
drinks_query_res = cur.fetchall()

In [ ]:

# Create a list of drink objects
for d in drinks_query_res:
    drinks = [{
        "drink_id": int(d[0]),
        "drink_name": d[1],
        "ingredients": d[2],
        "tags": d[3],
        "drink_description": f"{d[1]}: a {d[2]} cocktail made with {d[3]}"
        } 
        for d in drinks_query_res]

In [7]:
# Define vector db schema
class DrinkDoc(BaseDoc):
    drink_id: int
    drink_name: str
    ingredients: str
    tags: str
    embedding: NdArray[1536]


embeddings = OpenAIEmbeddings()

# # Embed description and create documents
# docs = DocList[DrinkDoc](
#     DrinkDoc(
#         drink_id=drink["drink_id"],
#         drink_name=drink["drink_name"],
#         ingredients=drink["ingredients"],
#         tags=drink["tags"],
#         embedding=embeddings.embed_query(drink['drink_description']),        
#     )
#     for drink in drinks
# )

In [55]:
# Create vector table in database
create_table_command = """
CREATE TABLE embeddings (
    id bigserial primary key,
    drink_id integer unique not null,
    drink_name text unique not null,
    ingredients text,
    tags text,
    embedding vector(1536)
);
"""
# Execute the SQL command
cur.execute(create_table_command)
# Commit the transaction
conn.commit()

In [ ]:

# Store documents with embeddings in the database
for doc in docs:
    # Insert embedding into database
    cur.execute("""
        INSERT INTO embeddings (drink_id, drink_name, ingredients, tags, embedding)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (drink_id) DO UPDATE
        SET embedding = EXCLUDED.embedding
    """, (doc.drink_id, doc.drink_name, doc.ingredients, doc.tags, doc.embedding))
    
conn.commit()

In [12]:
# Example similarity search
# <-> - L2 distance
# <#> - (negative) inner product
# <=> - cosine distance
# <+> - L1 distance
# <~> - Hamming distance (binary vectors)
# <%> - Jaccard distance (binary vectors)
def find_similar_drinks(query_text: str, limit: int = 10):
    query_embedding = embeddings.embed_query(query_text)
    
    cur.execute("""
        SELECT d.name, e.embedding <=> %s::vector as distance
        FROM embeddings e
        JOIN drinks d ON d.id = e.drink_id
        ORDER BY distance
        LIMIT %s
    """, (query_embedding, limit))
    
    return cur.fetchall()

In [15]:
# Test similarity search
similar_drinks = find_similar_drinks("Ingredients: bitter, strong alcohol taste, gin, herbal Mood: frustrated, party", limit=3)
print("Similar drinks:", similar_drinks)

Similar drinks: [('Dark and Moody', 0.12498629093170166), ('Bitter, Bitter, Bitter, Bitter', 0.12500778958270709), ('Bitter Tears', 0.12643039226531982)]


In [100]:
# Clean up
cur.close()
conn.close()